# Weather Preprocessing

In [1]:
%AddJar file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar

Starting download from file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar
Finished download of Emiasd-Flight-Data-Analysis.jar
Using cached version of Emiasd-Flight-Data-Analysis.jar


In [2]:
import org.apache.spark.sql.SparkSession
import com.flightdelay.config.{AppConfiguration, ConfigurationLoader}
import com.flightdelay.data.loaders.FlightDataLoader

//Env Configuration
val args: Array[String] = Array("jupyter")

val spark = SparkSession.builder()
  .config(sc.getConf)
  .getOrCreate()

// Rendre la session Spark implicite
implicit val session = spark
implicit val configuration: AppConfiguration = ConfigurationLoader.loadConfiguration(args)

//Set CheckPoint Dir
spark.sparkContext.setCheckpointDir(s"${configuration.common.output.basePath}/spark-checkpoints")
// Réduire les logs pour plus de clarté
spark.sparkContext.setLogLevel("WARN")

args = Array(jupyter)
spark = org.apache.spark.sql.SparkSession@325161e1
session = org.apache.spark.sql.SparkSession@325161e1
configuration = AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileConfig...


AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileConfig...

## 1. Chargement Weather Raw DataSet

In [3]:
val rawFlightsPath = s"${configuration.common.output.basePath}/Experience-Test/data/joined_exploded_data.parquet"
val rawFlightDF = spark.read.parquet(rawFlightsPath)

rawFlightDF.printSchema

root
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- DEST_WBAN: string (nullable = true)
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- ORIGIN_WBAN: string (nullable = true)
 |-- UTC_FL_DATE: date (nullable = true)
 |-- feature_utc_departure_hour_rounded: long (nullable = true)
 |-- is_delayed: integer (nullable = true)
 |-- origin_weather_WindDirection-7: double (nullable = true)
 |-- origin_weather_WindSpeedFlag-7: string (nullable = true)
 |-- origin_weather_DryBulbCelsiusFlag-7: string (nullable = true)
 |-- origin_weather_PressureChange-7: double (nullable = true)
 |-- origin_weather_DewPointCelsiusFlag-7: string (nullable = true)
 |-- origin_weather_DewPointFarenheit-7: double (nullable = true)
 |-- origin_weather_SeaLevelPressureFlag-7:

rawFlightsPath = /home/jovyan/work/output/Experience-Test/data/joined_exploded_data.parquet
rawFlightDF = [CRS_DEP_TIME: int, CRS_ELAPSED_TIME: double ... 650 more fields]


[CRS_DEP_TIME: int, CRS_ELAPSED_TIME: double ... 650 more fields]

In [4]:
val rawFlightsPath = s"${configuration.common.output.basePath}/common/data/raw_flights.parquet"
val rawFlightDF = spark.read.parquet(rawFlightsPath)

rawFlightDF.printSchema

root
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- ARR_DELAY_NEW: double (nullable = true)
 |-- CANCELLED: integer (nullable = true)
 |-- DIVERTED: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- WEATHER_DELAY: double (nullable = true)
 |-- NAS_DELAY: double (nullable = true)



rawFlightsPath = /home/jovyan/work/output/common/data/raw_flights.parquet
rawFlightDF = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 10 more fields]


[FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 10 more fields]

In [10]:
val rawWeatherPath = s"${configuration.common.output.basePath}/common/data/raw_weather.parquet"
val rawWeatherDF = spark.read.parquet(rawWeatherPath)

rawWeatherPath = /home/jovyan/work/output/common/data/raw_weather.parquet
rawWeatherDF = [WBAN: string, Date: string ... 42 more fields]


[WBAN: string, Date: string ... 42 more fields]

In [11]:
rawWeatherDF.count()

32631312

In [12]:
rawWeatherDF.printSchema

root
 |-- WBAN: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- StationType: integer (nullable = true)
 |-- SkyCondition: string (nullable = true)
 |-- SkyConditionFlag: string (nullable = true)
 |-- Visibility: double (nullable = true)
 |-- VisibilityFlag: string (nullable = true)
 |-- WeatherType: string (nullable = true)
 |-- WeatherTypeFlag: string (nullable = true)
 |-- DryBulbFarenheit: double (nullable = true)
 |-- DryBulbFarenheitFlag: string (nullable = true)
 |-- DryBulbCelsius: double (nullable = true)
 |-- DryBulbCelsiusFlag: string (nullable = true)
 |-- WetBulbFarenheit: double (nullable = true)
 |-- WetBulbFarenheitFlag: string (nullable = true)
 |-- WetBulbCelsius: double (nullable = true)
 |-- WetBulbCelsiusFlag: string (nullable = true)
 |-- DewPointFarenheit: double (nullable = true)
 |-- DewPointFarenheitFlag: string (nullable = true)
 |-- DewPointCelsius: double (nullable = true)
 |-- DewPointCelsiusFlag: string

In [13]:
val weatherColumns = Seq(
  "WBAN",
  "Date",
  "Time",
  "StationType",
  "SkyCondition",
  "Visibility",

  /**"most_critical_sky",
  "lowest_cloud_height",
  "ceiling",
  "num_cloud_layers",
  "cloud_risk_score",
  "has_overcast",
  "has_broken",
  "has_obscured",
  "is_clear",
  "has_low_ceiling",
  "visibility_miles",
  "visibility_km",
  "visibility_category",
  "visibility_risk_score",
  "is_low_visibility",
  "is_very_low_visibility",
  "visibility_normalized",
  "visibility_inverse",
  "weather_severity_index",
  "is_vfr_conditions",
  "is_ifr_conditions",
  "requires_cat_ii",
  "operations_risk_level"**/
)

val weatherData = rawWeatherDF
  .select(weatherColumns.head, weatherColumns.tail: _*)
weatherData.show(10)


weatherColumns = List(WBAN, Date, Time, StationType, SkyCondition, Visibility)
weatherData = [WBAN: string, Date: string ... 4 more fields]


+-----+--------+----+-----------+--------------------+----------+
| WBAN|    Date|Time|StationType|        SkyCondition|Visibility|
+-----+--------+----+-----------+--------------------+----------+
|03011|20130401|0000|          0|                 SCT|      10.0|
|03011|20130401|0015|          0|       SCT075 BKN085|      10.0|
|03011|20130401|0035|          0|       SCT075 OVC085|      10.0|
|03011|20130401|0055|          0|       BKN065 OVC075|      10.0|
|03011|20130401|0115|          0|SCT044 BKN055 OVC070|      10.0|
|03011|20130401|0135|          0|SCT047 BKN070 OVC110|      10.0|
|03011|20130401|0155|          0|SCT070 BKN090 OVC110|      10.0|
|03011|20130401|0215|          0|SCT055 BKN070 OVC090|      10.0|
|03011|20130401|0235|          0|SCT010 BKN021 OVC065|       7.0|
|03011|20130401|0255|          0|       BKN006 OVC012|       2.5|
+-----+--------+----+-----------+--------------------+----------+
only showing top 10 rows



[WBAN: string, Date: string ... 4 more fields]

# 2. Preprocessing

In [14]:
import com.flightdelay.data.preprocessing.weather.WeatherPreprocessingPipeline

val preprocessedWeatherData = WeatherPreprocessingPipeline.execute()


[Preprocessing] Weather Data Preprocessing Pipeline - Start

Loading raw data from parquet:
  - Path: /home/jovyan/work/output/common/data/raw_weather.parquet
  - Loaded 32631312 raw records

[STEP 2][DataCleaner] Weather Data Cleaning - Start

Original dataset: 32631312 records

Phase 1: Basic Cleaning
  - Current count: 32614082 records

Phase 2: Weather Time Normalization
  - Keeping only the closest record to HH:00 for each hour
  - Normalizing selected times to HH:00
  - Records before filtering: 32614082
  - Records after filtering: 11236398
  - Filtered out: 21377684 (66%)

Phase 3: Data Type Conversion
Conversion des types de données: WindDirection, DewPointFarenheit, WetBulbCelsius, DryBulbFarenheit, StationType, RelativeHumidity, Altimeter, DryBulbCelsius, WBAN, WindSpeed, Visibility, StationPressure, DewPointCelsius, Time, WetBulbFarenheit
  - Converting Date from YYYYMMDD to Date type
  - Current count: 11236398 records

Phase 4: Final Validation
  - Validation passed: 112

preprocessedWeatherData = [WBAN: string, Time: string ... 82 more fields]


[WBAN: string, Time: string ... 82 more fields]

# 3. Flight Category Index (Visibility and Ceiling)

In [15]:
//com.flightdelay.data.preprocessing.weather.VisibilityFeatures

val weatherColumns = Seq(
  "Visibility",
  "feature_ceiling",  
  "feature_visibility_miles",
  "feature_visibility_km",
  "feature_visibility_category",
  "feature_visibility_risk_score",
  "feature_is_low_visibility",
  "feature_is_very_low_visibility",
  "feature_visibility_normalized",
  "feature_visibility_inverse"
)

val weatherData = preprocessedWeatherData
  .select(weatherColumns.head, weatherColumns.tail: _*)
weatherData.show(10)


+----------+---------------+------------------------+---------------------+---------------------------+-----------------------------+-------------------------+------------------------------+-----------------------------+--------------------------+
|Visibility|feature_ceiling|feature_visibility_miles|feature_visibility_km|feature_visibility_category|feature_visibility_risk_score|feature_is_low_visibility|feature_is_very_low_visibility|feature_visibility_normalized|feature_visibility_inverse|
+----------+---------------+------------------------+---------------------+---------------------------+-----------------------------+-------------------------+------------------------------+-----------------------------+--------------------------+
|       9.0|           1500|                     0.9|               1.4481|                    IFR_LOW|                          4.0|                        1|                             1|                         0.09|        1.1111111111111112|
|      1

weatherColumns = List(Visibility, feature_ceiling, feature_visibility_miles, feature_visibility_km, feature_visibility_category, feature_visibility_risk_score, feature_is_low_visibility, feature_is_very_low_visibility, feature_visibility_normalized, feature_visibility_inverse)
weatherData = [Visibility: double, feature_ceiling: int ... 8 more fields]


[Visibility: double, feature_ceiling: int ... 8 more fields]

In [16]:
//com.flightdelay.data.preprocessing.weather.SkyConditionFeatures

val weatherColumns = Seq(
  "SkyCondition",
  "feature_most_critical_sky",  
  "feature_num_cloud_layers",
  "feature_cloud_risk_score",
    
  "feature_has_overcast",
  "feature_has_broken",
  "feature_is_low_visibility",
  "feature_has_obscured",
  "feature_is_clear",
    
  "feature_lowest_cloud_height",
  "feature_ceiling",
  "feature_has_low_ceiling"
    
)

val weatherData = preprocessedWeatherData
  .select(weatherColumns.head, weatherColumns.tail: _*)
weatherData.show(10)

+--------------------+-------------------------+------------------------+------------------------+--------------------+------------------+-------------------------+--------------------+----------------+---------------------------+---------------+-----------------------+
|        SkyCondition|feature_most_critical_sky|feature_num_cloud_layers|feature_cloud_risk_score|feature_has_overcast|feature_has_broken|feature_is_low_visibility|feature_has_obscured|feature_is_clear|feature_lowest_cloud_height|feature_ceiling|feature_has_low_ceiling|
+--------------------+-------------------------+------------------------+------------------------+--------------------+------------------+-------------------------+--------------------+----------------+---------------------------+---------------+-----------------------+
|       BKN015 OVC023|                      OVC|                       2|                     4.0|                true|              true|                        1|               false|  

weatherColumns = List(SkyCondition, feature_most_critical_sky, feature_num_cloud_layers, feature_cloud_risk_score, feature_has_overcast, feature_has_broken, feature_is_low_visibility, feature_has_obscured, feature_is_clear, feature_lowest_cloud_height, feature_ceiling, feature_has_low_ceiling)
weatherData = [SkyCondition: string, feature_most_critical_sky: string ... 10 more fields]


[SkyCondition: string, feature_most_critical_sky: string ... 10 more fields]

In [17]:
//com.flightdelay.data.preprocessing.weather.WeatherInteractionFeatures

val weatherColumns = Seq(
  "SkyCondition",
  "Visibility",
  "feature_flight_category",  
  "feature_weather_severity_index",  
  "feature_is_vfr_conditions",
  "feature_is_ifr_conditions",
  "feature_requires_cat_ii",
  "feature_operations_risk_level",
    
)

val weatherData = preprocessedWeatherData
  .select(weatherColumns.head, weatherColumns.tail: _*)
weatherData.show(10)

+--------------------+----------+------------------------------+-------------------------+-------------------------+-----------------------+-----------------------------+
|        SkyCondition|Visibility|feature_weather_severity_index|feature_is_vfr_conditions|feature_is_ifr_conditions|feature_requires_cat_ii|feature_operations_risk_level|
+--------------------+----------+------------------------------+-------------------------+-------------------------+-----------------------+-----------------------------+
|       BKN015 OVC023|       9.0|                           5.0|                        0|                        1|                      1|                            3|
|              OVC013|      10.0|                           3.4|                        0|                        1|                      0|                            2|
|              OVC023|       8.0|                           5.0|                        0|                        1|                      1|     

weatherColumns = List(SkyCondition, Visibility, feature_weather_severity_index, feature_is_vfr_conditions, feature_is_ifr_conditions, feature_requires_cat_ii, feature_operations_risk_level)
weatherData = [SkyCondition: string, Visibility: double ... 5 more fields]


[SkyCondition: string, Visibility: double ... 5 more fields]

In [18]:
//com.flightdelay.data.preprocessing.weather.WeatherTypeFeatureGenerator

val weatherColumns = Seq(
  "SkyCondition",
  "Visibility",
    
  "intensity_heavy",  
  "intensity_light",
  "weather_intensity",
  "has_thunderstorm",
  "has_freezing_precip",
  "has_freezing",
  "has_precipitation",
  "has_obscuration",
  "has_visible_moisture",
  "has_hazardous",
  "extracted_codes",
  "weather_hazard_level",

  "Icing_Risk_Flag",
  "Icing_Risk_Level",
    
    
)

val weatherData = preprocessedWeatherData
  .select(weatherColumns.head, weatherColumns.tail: _*)
weatherData.show(10)

+--------------------+----------+---------------+---------------+-----------------+----------------+-------------------+------------+-----------------+---------------+--------------------+-------------+---------------+--------------------+---------------+----------------+
|        SkyCondition|Visibility|intensity_heavy|intensity_light|weather_intensity|has_thunderstorm|has_freezing_precip|has_freezing|has_precipitation|has_obscuration|has_visible_moisture|has_hazardous|extracted_codes|weather_hazard_level|Icing_Risk_Flag|Icing_Risk_Level|
+--------------------+----------+---------------+---------------+-----------------+----------------+-------------------+------------+-----------------+---------------+--------------------+-------------+---------------+--------------------+---------------+----------------+
|       BKN015 OVC023|       9.0|              0|              0|         moderate|               0|                  0|           0|                0|              0|              

weatherColumns = List(SkyCondition, Visibility, intensity_heavy, intensity_light, weather_intensity, has_thunderstorm, has_freezing_precip, has_freezing, has_precipitation, has_obscuration, has_visible_moisture, has_hazardous, extracted_codes, weather_hazard_level, Icing_Risk_Flag, Icing_Risk_Level)
weatherData = [SkyCondition: string, Visibility: double ... 14 more fields]


[SkyCondition: string, Visibility: double ... 14 more fields]

# 4. Wind Severity and Gusts 

In [23]:
//com.flightdelay.data.preprocessing.weather.WindFeatureGenerator

val weatherColumns = Seq(
 "WindSpeed",
 "WindDirection",
 "ValueForWindCharacter"    
)

val weatherData = preprocessedWeatherData
  .select(weatherColumns.head, weatherColumns.tail: _*)
weatherData.show(10)

//weatherData.select("ValueForWindCharacter").distinct().show(10)

+---------+-------------+---------------------+
|WindSpeed|WindDirection|ValueForWindCharacter|
+---------+-------------+---------------------+
|      9.0|        120.0|                     |
|     21.0|        360.0|                   30|
|      3.0|        100.0|                     |
|      6.0|        200.0|                     |
|     10.0|        130.0|                     |
|      5.0|        200.0|                     |
|      9.0|         80.0|                     |
|      7.0|         60.0|                     |
|     11.0|         70.0|                     |
|     24.0|         10.0|                   30|
+---------+-------------+---------------------+
only showing top 10 rows



weatherColumns = List(WindSpeed, WindDirection, ValueForWindCharacter)
weatherData = [WindSpeed: double, WindDirection: double ... 1 more field]


[WindSpeed: double, WindDirection: double ... 1 more field]

# 5. Convective and Storm Activity

In [ ]:
//com.flightdelay.data.preprocessing.weather.WindFeatureGenerator
// ?????

val weatherColumns = Seq(
 "WindSpeed",
 "WindDirection",
 "ValueForWindCharacter"    
)

val weatherData = preprocessedWeatherData
  .select(weatherColumns.head, weatherColumns.tail: _*)
//weatherData.show(10)